<a href="https://colab.research.google.com/github/IgorBaratta/FEniCSxCourse/blob/main/Problem8_HPC/HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DOLFINx in Parallel with MPI

To run the programs in this section, first you will need to install the dolfinx and the mpi4py library:

In [ ]:
try:
  import dolfinx
except ImportError:
  !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/7f220b6/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
  import dolfinx

## First parallel program

In [ ]:
%%writefile hello-world.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
print(f"Hello world from rank {comm.rank} of {comm.size}")

Next we see how we can use the mpirun program to execute the above python code using 4 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [ ]:
! mpirun --allow-run-as-root --oversubscribe -np 8 python hello-world.py

## Point-to-Point Communication

In [ ]:
%%writefile p2p.py

from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
assert comm.size == 2
rank = comm.rank

if rank == 0:
    data = numpy.arange(10, dtype=int)
    comm.Send(data, dest=1)
    print(f"Process 0 sent {data} to process 1")
elif rank == 1:
    data = numpy.zeros(10, dtype=int)
    comm.Recv(data, source=0)
    print(f"Process 1 received {data} from process 0")


In [ ]:
!mpirun -n 2 --allow-run-as-root --oversubscribe python3 p2p.py

MPI can do a lot more than this (https://mpi4py.readthedocs.io/en/stable/tutorial.html). The key points are:
 - N identical versions of your program run, one on each process (rank). Each process takes different paths through the program depending on its rank.
 - Processes (and hence their memory) are totally independent.
 - Communication between processes is must be manually performed by the programmer (explicit).

## Creating a mesh in parallel
DOLFINx abstracts most of the difficult aspects of distributing your problem across the MPI communicator away from the user. 
Let's have a look at how to create and write a mesh in parallel: 

In [ ]:
%%writefile mesh.py

from mpi4py import MPI
import dolfinx
from dolfinx.io import XDMFFile

comm = MPI.COMM_WORLD

mesh = dolfinx.mesh.create_unit_square(comm, 20, 20)
DG = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))
cell_value = dolfinx.fem.Function(DG)
cell_value.x.array[:] = comm.rank

# Save mesh and 
with XDMFFile(comm, f"mesh_{comm.size}.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(cell_value)

In [ ]:
! mpirun -n 2 --allow-run-as-root --oversubscribe python3 mesh.py

Try changing the number of processes and visualizing the mesh.

## Managing data in parallel: the IndexMap

The IndexMap represents the distribution index arrays across processes. An index array is a contiguous collection of N+1 indices `[0, 1, . . ., N]` that are distributed across M processes. On a given process, the IndexMap stores a portion of the index set using local indices `[0, 1, . . . , n]`, and a map from the local indices to a unique global index.

Let's have a look at an example. 
How many cells are owned by a given process?

In [ ]:
%%writefile index_map.py

from mpi4py import MPI
import dolfinx
from dolfinx.io import XDMFFile

comm = MPI.COMM_WORLD
mesh = dolfinx.mesh.create_unit_square(comm, 20, 20)
tdim = mesh.topology.dim
idx_map = mesh.topology.index_map(tdim)

N = idx_map.size_global
n = idx_map.size_local
print(f"Process {comm.rank} owns {n} cells of {N}")

comm.Barrier()
print("")
comm.Barrier()

vmap = mesh.topology.index_map(0)
print(f"Process {comm.rank} owns {vmap.size_local} vertices of {vmap.size_global}")



In [ ]:
! mpirun -n 2 --allow-run-as-root --oversubscribe python3 index_map.py

## Function and FunctionSpaces

The distribution of a vector of function across the MPI process is also managed by an IndexMap and it follows the distribution of the mesh. 

Consider a continuous first-order Lagrange space over a mesh. The degrees of freedom of this space are associated with the vertices of the mesh.

A function in DOLFINx contains memory (an array) in which the expansion coefficients ($u_i$) of the finite element basis ($\phi_i$) can be stored:

$$
u_h = \sum_{i = 0}^{N-1} \phi_i u_i
$$




In [ ]:
%%writefile function.py

from mpi4py import MPI
import dolfinx
from dolfinx.io import XDMFFile

comm = MPI.COMM_WORLD
mesh = dolfinx.mesh.create_unit_square(comm, 20, 20)
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
u = dolfinx.fem.Function(V)

map = V.dofmap.index_map

print(f"Process {comm.rank} owns {map.size_local} dofs of {map.size_global}")

comm.Barrier()
print("")
comm.Barrier()

print(f"Local size {u.x.array.size} in process {comm.rank}")

# The size of local vector includes owned dofs and ghosts
assert u.x.array.size ==  map.size_local + map.num_ghosts

In [ ]:
! mpirun -n 4 --allow-run-as-root --oversubscribe python3 function.py

## Scattering values

Let’s say we want to change the expansion coefficient $u_i$ (local numbering) each processes to have a value equal to the MPI rank + 1 of the owning process. For consistency we need this change to be reflected in two places:

 - In the memory of the process that owns the degree of freedom.
 - In the memory of the process (if any) that has the degree of freedom as a ghost.


How do we do that?

In [ ]:
%%writefile scatter.py

from mpi4py import MPI
import dolfinx
from dolfinx.io import XDMFFile

comm = MPI.COMM_WORLD
mesh = dolfinx.mesh.create_unit_square(comm, 5, 5)
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
u = dolfinx.fem.Function(V)

u.x.array[:] = comm.rank

for i in range(comm.size):
    if comm.rank == i:
      print(f"Data on process {comm.rank}: {u.x.array}")
    comm.Barrier()

comm.Barrier()
print("")
comm.Barrier()

# Scatter values to sharing process
u.x.scatter_forward()

for i in range(comm.size):
    if comm.rank == i:
      print(f"Data on process {comm.rank}: {u.x.array}")
    comm.Barrier()

In [ ]:
! mpirun -n 4 --allow-run-as-root --oversubscribe python3 scatter.py

## Assembling vectors in parallel

When we call `dolfinx.fem.assemble_vector(L)` the following happens:
 - Each process calculates the cell tensors bT for cells that it owns.
 - It assembles (adds) the result into its local array.

At this point no parallel communication has taken place! The vector is in an inconsistent state, a contributiong to a degree of freedom might have taken place in another process.

To update the values we use:
```
x.scatter_rev(ScatterMode.add)
```

In [ ]:
%%writefile linear_form.py

from mpi4py import MPI
from petsc4py import PETSc
import dolfinx
import ufl

comm = MPI.COMM_WORLD


comm = MPI.COMM_WORLD
mesh = dolfinx.mesh.create_unit_square(comm, 2, 2, ghost_mode=dolfinx.mesh.GhostMode.none)
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
u = dolfinx.fem.Function(V)
v = ufl.TestFunction(V)

L = ufl.inner(5.0, v)*ufl.dx

b = dolfinx.fem.assemble_vector(dolfinx.fem.form(L))

if comm.rank == 0:
    print("Before scattering")

for i in range(comm.size):
    if comm.rank == i:
      print(f"Data on process {comm.rank}: {b.array}")
    comm.Barrier()

b.scatter_reverse(dolfinx.la.ScatterMode.add)

if comm.rank == 0:
    print("\nAfter scattering")

for i in range(comm.size):
    if comm.rank == i:
      print(f"Data on process {comm.rank}: {b.array}")
    comm.Barrier()

In [ ]:
! mpirun -n 2 --allow-run-as-root --oversubscribe python3 linear_form.py

## Assembling matrices in parallel


## Solving a linear system in parallel

## Parallel Scaling

High performance computing has two common notions of scalability:

 - Strong scaling is defined as how the solution time varies with the number of processors for a fixed total problem size.
 - Weak scaling is defined as how the solution time varies with the number of processors for a fixed problem size per processor.

# Homework 8

Scalability study: [poisson](https://github.com/IgorBaratta/FEniCSxCourse/blob/main/Problem8_HPC/poisson.py)
